이번 노트북에서는 colab 에서 제공하는 전분가를 위한 빠른 시작을 사용하여 TPU 모델을 재구성 하였다.

아래의 코드를 제공하는 이유는 연구자들이 code를 확장하는데 도움을 주고자 하여 만들었다.

compare 도구를 사용하여 "41_TF2_MNIST_quick_start_for_expert.py"와 "42_TF2_MNIST_quick_start_for_expert_TPU.py"를 비교하여 보자.

In [6]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Model

import numpy as np

import os
import distutils
if distutils.version.LooseVersion(tf.__version__) <= '2.0':
    raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/fashion_mnist.ipynb')

resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.experimental.TPUStrategy(resolver)


print(tf.__version__)

learning_rate = 0.001
training_epochs = 15
batch_size = 32

## MNIST Dataset #########################################################
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()    
##########################################################################

train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

# train_images, test_images = train_images / 255.0, test_images / 255.0

# 채널 차원을 추가합니다.
# train_images = train_images[..., tf.newaxis]
# test_images = test_images[..., tf.newaxis]
    
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(
                buffer_size=10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

class MNISTModel(tf.keras.Model):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1      = Conv2D(filters=32, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)
        self.pool1      = MaxPool2D(padding='SAME')
        self.conv2      = Conv2D(filters=64, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)
        self.pool2      = MaxPool2D(padding='SAME')
        self.conv3      = Conv2D(filters=128, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)
        self.pool3      = MaxPool2D(padding='SAME')
        self.pool3_flat = Flatten()
        self.dense4     = Dense(units=256, activation=tf.nn.relu)
        self.drop4      = Dropout(rate=0.4)
        self.dense5     = Dense(units=10, activation='softmax')
    def call(self, inputs, training=False):
        net = self.conv1(inputs)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.pool3(net)
        net = self.pool3_flat(net)
        net = self.dense4(net)
        net = self.drop4(net)
        net = self.dense5(net)
        return net

with strategy.scope():
    model = MNISTModel()

    loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

    optimizer = tf.keras.optimizers.Adam()

    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

    test_loss = tf.keras.metrics.Mean(name='test_loss')
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

    @tf.function
    def train_step(images, labels):
        with tf.GradientTape() as tape:
            predictions = model(images)
            loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_accuracy(labels, predictions)

    @tf.function
    def test_step(images, labels):
        predictions = model(images)
        t_loss = loss_object(labels, predictions)

        test_loss(t_loss)
        test_accuracy(labels, predictions)

# EPOCHS = 5

for epoch in range(training_epochs):
    for images, labels in train_dataset:
        train_step(images, labels)

    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)

    template = 'epoch: {}, loss: {}, accuracy: {}, test loss: {}, test accuracy: {}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))


INFO:tensorflow:Initializing the TPU system: grpc://10.78.242.250:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.78.242.250:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


2.3.0
epoch: 1, loss: 0.11426263302564621, accuracy: 96.39166259765625, test loss: 0.05454486235976219, test accuracy: 98.31999969482422
epoch: 2, loss: 0.07679859548807144, accuracy: 97.6050033569336, test loss: 0.04288095980882645, test accuracy: 98.61500549316406
epoch: 3, loss: 0.060273706912994385, accuracy: 98.12222290039062, test loss: 0.0389101468026638, test accuracy: 98.77333068847656
epoch: 4, loss: 0.0504249632358551, accuracy: 98.42791748046875, test loss: 0.036348436027765274, test accuracy: 98.86750030517578
epoch: 5, loss: 0.04367636516690254, accuracy: 98.6336669921875, test loss: 0.035966698080301285, test accuracy: 98.91200256347656
epoch: 6, loss: 0.038742754608392715, accuracy: 98.78194427490234, test loss: 0.034708693623542786, test accuracy: 98.95832824707031
epoch: 7, loss: 0.034581735730171204, accuracy: 98.91500091552734, test loss: 0.03355927765369415, test accuracy: 98.99713897705078
epoch: 8, loss: 0.031521815806627274, accuracy: 99.00979614257812, test los